In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def generate_wave(period_frames,length_frames):
    x = np.arange(length_frames)
    sin1 = np.sin(x/period_frames*2*np.pi)
    sin2 = np.sin((x-1)/period_frames*2*np.pi)
    wave = np.abs(sin1-sin2)
    wave += np.random.randn(length_frames)*0.01
    return wave

def generate_random_repeating_wave(period_frames,length_frames):
    repeats = int(np.ceil(length_frames/period_frames)) 
    chunk = np.abs(np.random.randn(period_frames))
    wave = np.tile(chunk,repeats)[:length_frames]
    wave += np.random.randn(length_frames)*0.3
    return wave

def generate_random_half_repeating_wave(period_frames,length_frames):
    repeats = int(np.ceil(length_frames/period_frames)) 
    chunk = np.abs(np.random.randn(period_frames))
    wave = np.tile(chunk,repeats)[:length_frames]
    wave[:length_frames//10] = np.random.randn(length_frames//10)
    wave += np.random.randn(length_frames)*0.3
    return wave
    


In [ ]:
fps             = 30
periods_seconds = np.array( [0.5, 2] ) 
periods_frames  = np.round(fps * periods_seconds).astype("int")

p1,p2 =  periods_frames[:]

wave_length = p2*4

period_list = list(range(p1,p2,3))
plt.figure(figsize=(15,30))

wave_list = []

for i,p in enumerate(period_list):
    
#     wave = generate_random_repeating_wave(p,wave_length)
#     wave = generate_wave(p,wave_length)
    wave = generate_random_half_repeating_wave(p,wave_length)
    wave_list.append((p,wave))
    
    w=3
    plt.subplot(len(period_list)//w+1,w,i+1)
    plt.plot(wave)
    plt.title(f"Period: {p}")




In [ ]:
plt.figure(figsize=(15,30))
p_max = 60
length = 120

harmonic_weighting = np.zeros( (p_max+1,length) )

# harmonic_weighting[0,0] = 1
for i in range(1,p_max+1):
    for j in range(i,length,i):
        harmonic_weighting[i,j] = 1/len(range(i,length,i))
        
for i,(period,X) in enumerate(wave_list):
    length = len(X)
    
    X = X - X.mean()
    
    #apply a windowing function to the signal so the ends taper to zero
    X_windowed = X #* np.blackman(length)

    #calculate the value for correlating the signal with itself for every possible shift
    correlation = np.correlate(X_windowed,X_windowed,"same")

    
    correlation /= correlation[length//2]
    
    #the correlation is symetric so just take the second half
    correlation = correlation[len(correlation)//2:]
    
    line = 1 - 0.5*np.arange(len(correlation))/len(correlation)
    
    correlation = correlation/line
    
    harmonic_mean = (correlation * harmonic_weighting).sum(axis=1)
    
    best = np.argmax(harmonic_mean)
    w=3
    plt.subplot(len(period_list)//w+1,w,i+1)
    plt.plot([period,period],[0,1])
    plt.plot(correlation)
    plt.plot(harmonic_mean)
    plt.plot(best,harmonic_mean[best],"x")
    
    plt.title(f"Period: {period}")
    
    